In [9]:
import tensorflow as tf
import pandas as pd
tf.__version__

'2.13.0'

In [10]:
# Read data to train on
df = pd.read_csv('../input_data/CSVs/Bias_correction_ucl.csv')

# Drop the date column from the dataframe
df.drop('Date', inplace=True, axis=1)
# Drop any columns with null values
df.dropna(inplace=True)
df

,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
0,1.0,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,...,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1,21.2
1,2.0,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,0.225508,...,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5,22.5
2,3.0,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,0.209344,...,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1,23.9
3,4.0,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,0.216372,...,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7,24.3
4,5.0,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,0.151407,...,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2,22.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,21.0,23.1,17.8,24.688997,78.261383,27.812697,18.303014,6.603253,9.614074,0.052902,...,0.0,0.0,0.0,37.5507,127.040,26.2980,0.5721,4456.024414,27.6,17.7
7746,22.0,22.5,17.4,30.094858,83.690018,26.704905,17.814038,5.768083,82.146707,0.066461,...,0.0,0.0,0.0,37.5102,127.086,21.9668,0.1332,4441.803711,28.0,17.1
7747,23.0,23.3,17.1,26.741310,78.869858,26.352081,18.775678,6.148918,72.058294,0.030034,...,0.0,0.0,0.0,37.5372,126.891,15.5876,0.1554,4443.313965,28.3,18.1
7748,24.0,23.3,17.7,24.040634,77.294975,27.010193,18.733519,6.542819,47.241457,0.035874,...,0.0,0.0,0.0,37.5237,126.909,17.2956,0.2223,4438.373535,28.6,18.8


In [11]:
# Create the target and feature datasets from the dataframe

# The target will contain the next max and min temps to predict
target = df[['Next_Tmax', 'Next_Tmin']]
# And the feature set will have all data bar the features to predict
features = df.drop(['Next_Tmax', 'Next_Tmin'], axis=1)

In [12]:
from sklearn.preprocessing import MinMaxScaler
# Rescale the dataset to between 0 and 1
scaler = MinMaxScaler()
feature_array = scaler.fit_transform(features)
features = pd.DataFrame(feature_array, columns=features.columns)

In [13]:
# Create a new model to form the temperature predictor
model = tf.keras.Sequential()
# Add an input layer with an amount of inputs equal to the number of columns in the feature set
model.add(tf.keras.layers.InputLayer(input_shape=(features.shape[1],), name='Input_layer'))
# Add a dense layer to create the 2 necessary outputs that represent the temperature predictions
model.add(tf.keras.layers.Dense(2, name='Output_layer'))

In [14]:
# Use RMSprop as the optimiser function, with loss calculated by the mean square error
model.compile(tf.optimizers.RMSprop(0.001), loss='mse')

# Add a callback for tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [15]:
# Fit the model to the training data, with 50 runs through the dataset
model.fit(x=features.to_numpy(), y=target.to_numpy(), epochs=50, callbacks=[tensorboard_callback])

Epoch 1/50
238/238 [==============================] - 1s 1ms/step - loss: 683.1458
Epoch 2/50
238/238 [==============================] - 0s 1ms/step - loss: 576.9147
Epoch 3/50
238/238 [==============================] - 0s 1ms/step - loss: 480.2981
Epoch 4/50
238/238 [==============================] - 0s 1ms/step - loss: 393.2728
Epoch 5/50
238/238 [==============================] - 0s 1ms/step - loss: 315.8923
Epoch 6/50
238/238 [==============================] - 0s 1ms/step - loss: 248.0395
Epoch 7/50
238/238 [==============================] - 0s 1ms/step - loss: 189.7799
Epoch 8/50
238/238 [==============================] - 0s 1ms/step - loss: 141.1825
Epoch 9/50
238/238 [==============================] - 0s 1ms/step - loss: 101.9670
Epoch 10/50
238/238 [==============================] - 0s 1ms/step - loss: 72.4541
Epoch 11/50
238/238 [==============================] - 0s 1ms/step - loss: 51.7156
Epoch 12/50
238/238 [==============================] - 0s 2ms/step - loss: 37.8825
Epoc

In [16]:
# Evaluate the model on training data
loss = model.evaluate(features.to_numpy(), target.to_numpy())
# Output the loss on the tested training data
print('loss:', loss)

238/238 [==============================] - 0s 1ms/step - loss: 3.5137
loss: 3.513746976852417
